#### Simple Gen AI APP Using Langchain


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [6]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [17]:
loader = WebBaseLoader(
    "https://medium.com/@kelvin.lu.au/tracing-how-langchain-works-behind-the-scenes-c17425ed5d1d"
)
loader

In [18]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://medium.com/@kelvin.lu.au/tracing-how-langchain-works-behind-the-scenes-c17425ed5d1d', 'title': 'Tracing How LangChain Works Behind the Scenes | by Kelvin Lu | Medium', 'description': 'Have you ever wondered how LangChain works its magic? When you ask LangChain a question, it seems to just know the answer. But how does it do that? Is the answer trustworthy, or is it just a…', 'language': 'en'}, page_content='Tracing How LangChain Works Behind the Scenes | by Kelvin Lu | MediumOpen in appSign upSign inWriteSign upSign inTracing How LangChain Works Behind the ScenesKelvin Lu·Follow7 min read·Jun 26, 2023--ListenShareHave you ever wondered how LangChain works its magic? When you ask LangChain a question, it seems to just know the answer. But how does it do that? Is the answer trustworthy, or is it just a hallucination? How to tell if a result results from prudent reasoning or a devilish joke?René DescartesFour hundred years ago, French mathematician a

In [19]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [20]:
documents

[Document(metadata={'source': 'https://medium.com/@kelvin.lu.au/tracing-how-langchain-works-behind-the-scenes-c17425ed5d1d', 'title': 'Tracing How LangChain Works Behind the Scenes | by Kelvin Lu | Medium', 'description': 'Have you ever wondered how LangChain works its magic? When you ask LangChain a question, it seems to just know the answer. But how does it do that? Is the answer trustworthy, or is it just a…', 'language': 'en'}, page_content='Tracing How LangChain Works Behind the Scenes | by Kelvin Lu | MediumOpen in appSign upSign inWriteSign upSign inTracing How LangChain Works Behind the ScenesKelvin Lu·Follow7 min read·Jun 26, 2023--ListenShareHave you ever wondered how LangChain works its magic? When you ask LangChain a question, it seems to just know the answer. But how does it do that? Is the answer trustworthy, or is it just a hallucination? How to tell if a result results from prudent reasoning or a devilish joke?René DescartesFour hundred years ago, French mathematician a

In [21]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [22]:
from langchain_community.vectorstores import FAISS

vectorstoredb = FAISS.from_documents(documents, embeddings)

In [23]:
vectorstoredb

In [24]:
## Query From a vector db
query = "LangSmith has two usage limits: total traces and extended"
result = vectorstoredb.similarity_search(query)
result

[Document(id='22e3a6cc-a769-489f-b440-b2894db47b9d', metadata={'source': 'https://medium.com/@kelvin.lu.au/tracing-how-langchain-works-behind-the-scenes-c17425ed5d1d', 'title': 'Tracing How LangChain Works Behind the Scenes | by Kelvin Lu | Medium', 'description': 'Have you ever wondered how LangChain works its magic? When you ask LangChain a question, it seems to just know the answer. But how does it do that? Is the answer trustworthy, or is it just a…', 'language': 'en'}, page_content='tracing can also be used to build trust with users. By showing users how the system works, we can help them understand that the system is not making decisions in a black box.In conclusion, tracing is a valuable tool for engineers who are working with GAI systems. I hope you find spending time on it worthwhile.LangchainLlmGenerative Ai ToolsOpenAIChatGPT----FollowWritten by Kelvin Lu958 Followers·291 FollowingSydney-based ML engineer / data engineer @ InteliaFollowNo responses yetHelpStatusAboutCareersP

In [25]:
result[0].page_content

'tracing can also be used to build trust with users. By showing users how the system works, we can help them understand that the system is not making decisions in a black box.In conclusion, tracing is a valuable tool for engineers who are working with GAI systems. I hope you find spending time on it worthwhile.LangchainLlmGenerative Ai ToolsOpenAIChatGPT----FollowWritten by Kelvin Lu958 Followers·291 FollowingSydney-based ML engineer / data engineer @ InteliaFollowNo responses yetHelpStatusAboutCareersPressBlogPrivacyTermsText to speechTeams'

## Document and Retriever chain


In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [28]:
## Retrieval Chain, Document chain
# Here we create a chain that combines retrieved documents with a prompt template
# to generate responses from the LLM

# Import required components
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Create a prompt template that instructs the LLM to answer based only on provided context
prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

# Create a chain that:
# 1. Takes the retrieved documents
# 2. Combines them into the prompt template
# 3. Sends to the LLM for response generation
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11b8551d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11b861050>, root_client=<openai.OpenAI object at 0x11b84d550>, root_async_client=<openai.AsyncOpenAI object at 0x11b855390>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docume

In [32]:
from langchain_core.documents import Document

document_chain.invoke(
    {
        "input": "LangSmith has two usage limits: total traces and extended",
        "context": [
            Document(
                page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. "
            )
        ],
    }
)

"LangSmith's usage limits include two key metrics: total traces and extended traces. These are the metrics tracked on their usage graph."

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.


In [33]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [34]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [35]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11a99ded0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatOpe

In [36]:
## Get the response form the LLM
response = retrieval_chain.invoke(
    {"input": "LangSmith has two usage limits: total traces and extended"}
)
response["answer"]

'What is the purpose of tracing in the context of LangChain and Generative AI systems? \n\nThe purpose of tracing in the context of LangChain and Generative AI systems is to improve debugging and performance, ensure transparency, and build trust with users. Tracing allows users to understand how the system operates internally, making it clear that decisions are not made in a "black box." It helps identify areas where improvements can be made to make the system more efficient and effective. Additionally, it provides detailed information on the system\'s operations, which can be valuable for engineers working with GAI systems.'

In [37]:
response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='22e3a6cc-a769-489f-b440-b2894db47b9d', metadata={'source': 'https://medium.com/@kelvin.lu.au/tracing-how-langchain-works-behind-the-scenes-c17425ed5d1d', 'title': 'Tracing How LangChain Works Behind the Scenes | by Kelvin Lu | Medium', 'description': 'Have you ever wondered how LangChain works its magic? When you ask LangChain a question, it seems to just know the answer. But how does it do that? Is the answer trustworthy, or is it just a…', 'language': 'en'}, page_content='tracing can also be used to build trust with users. By showing users how the system works, we can help them understand that the system is not making decisions in a black box.In conclusion, tracing is a valuable tool for engineers who are working with GAI systems. I hope you find spending time on it worthwhile.LangchainLlmGenerative Ai ToolsOpenAIChatGPT----FollowWritten by Kelvin Lu958 Followers·291 FollowingSydney-based

In [38]:
response["context"]

[Document(id='22e3a6cc-a769-489f-b440-b2894db47b9d', metadata={'source': 'https://medium.com/@kelvin.lu.au/tracing-how-langchain-works-behind-the-scenes-c17425ed5d1d', 'title': 'Tracing How LangChain Works Behind the Scenes | by Kelvin Lu | Medium', 'description': 'Have you ever wondered how LangChain works its magic? When you ask LangChain a question, it seems to just know the answer. But how does it do that? Is the answer trustworthy, or is it just a…', 'language': 'en'}, page_content='tracing can also be used to build trust with users. By showing users how the system works, we can help them understand that the system is not making decisions in a black box.In conclusion, tracing is a valuable tool for engineers who are working with GAI systems. I hope you find spending time on it worthwhile.LangchainLlmGenerative Ai ToolsOpenAIChatGPT----FollowWritten by Kelvin Lu958 Followers·291 FollowingSydney-based ML engineer / data engineer @ InteliaFollowNo responses yetHelpStatusAboutCareersP